In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/hafsa/')

In [ ]:
cd /content/drive/MyDrive/hafsa

/content


In [ ]:
# Cell 1 — Imports
# Comment: Standard libraries for file I/O, XML parsing, data handling, and JSONL writing.

from pathlib import Path
import os
import json
import xml.etree.ElementTree as ET
from dataclasses import dataclass, asdict
from typing import Optional, List, Tuple
import pandas as pd



In [ ]:
# Cell 2 — Locate input XML(s) & configure paths
# Comment: Auto-detect a valid UniTime XML in /mnt/data. Prefer known names; else first valid.

DATA_DIR = Path("/mnt/data/content/pu-spr07-cs.xml")
OUTPUT_DIR = Path("/mnt/data/unittime_goal1_out")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# List visible .xml files to help debugging path issues
print("XML candidates in /mnt/data:", [p.name for p in DATA_DIR.glob("*.xml")])

def is_unittime_xml(path: Path) -> bool:
    try:
        root = ET.parse(str(path)).getroot()
        return root.tag == "timetable" and root.get("nrDays") is not None and root.get("slotsPerDay") is not None
    except Exception:
        return False

# Prefer common filename if available
preferred_names = ["/content/pu-spr07-cs.xml", "/content/pu-spr07-cs.xml"]
INPUT_XML = None

for name in preferred_names:
    cand = DATA_DIR / name
    if cand.exists() and is_unittime_xml(cand):
        INPUT_XML = cand
        break

if INPUT_XML is None:
    # Fallback: pick the first valid UniTime timetable XML
    for cand in DATA_DIR.glob("*.xml"):
        if is_unittime_xml(cand):
            INPUT_XML = cand
            break

if INPUT_XML is None:
    raise FileNotFoundError("No valid UniTime timetable XML found in /mnt/data (e.g., pu-spr07-cs.xml).")

print(f"Using XML: {INPUT_XML}")
print(f"Output dir: {OUTPUT_DIR}")


XML candidates in /mnt/data: []
Using XML: /content/pu-spr07-cs.xml
Output dir: /mnt/data/unittime_goal1_out


In [ ]:
# Cell 3 — Dataclasses & helpers
# Comment: Define record structures and safe parsers (ints/floats/locations/booleans).

@dataclass
class Room:
    room_id: str
    capacity: Optional[int]
    loc_x: Optional[int]
    loc_y: Optional[int]
    sharing_pattern: Optional[str] = None
    sharing_unit: Optional[int] = None
    initiative: Optional[str] = None
    term: Optional[str] = None

@dataclass
class ClassRec:
    class_id: str
    offering: Optional[str]
    config: Optional[str]
    subpart: Optional[str]
    parent_class_id: Optional[str]
    class_limit: Optional[int]
    min_limit: Optional[int] = None
    max_limit: Optional[int] = None
    nr_rooms: Optional[int] = None
    scheduler_dept: Optional[str] = None
    committed: Optional[bool] = None
    dates_mask: Optional[str] = None

@dataclass
class CandidateTime:
    class_id: str
    days: str
    start: int
    length: int
    pref: Optional[float]

@dataclass
class CandidateRoom:
    class_id: str
    room_id: str
    pref: Optional[float]

@dataclass
class ClassInstructor:
    class_id: str
    instructor_id: str

def parse_bool(s: Optional[str]) -> Optional[bool]:
    if s is None:
        return None
    return s.lower() == "true"

def parse_location(s: Optional[str]) -> Tuple[Optional[int], Optional[int]]:
    if not s or "," not in s:
        return None, None
    try:
        xs, ys = s.split(",", 1)
        return int(xs), int(ys)
    except Exception:
        return None, None

def attr_int(elem, key, default=None):
    v = elem.get(key)
    try:
        return int(v) if v is not None else default
    except Exception:
        return default

def attr_float(elem, key, default=None):
    v = elem.get(key)
    try:
        return float(v) if v is not None else default
    except Exception:
        return default


In [ ]:
# Cell 4 — Parse XML → raw lists
# Comment: Walk the XML and collect Rooms, Classes, Times, Room prefs, Instructors.

tree = ET.parse(str(INPUT_XML))
root = tree.getroot()

initiative = root.get("initiative")
term = root.get("term")
nr_days = attr_int(root, "nrDays", default=7)
slots_per_day = attr_int(root, "slotsPerDay", default=288)

rooms: List[Room] = []
classes: List[ClassRec] = []
cand_times: List[CandidateTime] = []
cand_rooms: List[CandidateRoom] = []
class_instructors: List[ClassInstructor] = []

# Rooms
rooms_node = root.find("rooms")
if rooms_node is not None:
    for r in rooms_node.findall("room"):
        rid = r.get("id")
        cap = attr_int(r, "capacity")
        loc_x, loc_y = parse_location(r.get("location"))
        pattern, unit = None, None
        sharing = r.find("sharing")
        if sharing is not None:
            pat = sharing.find("pattern")
            if pat is not None:
                pattern = (pat.text or "").strip()
                unit = attr_int(pat, "unit")
        rooms.append(Room(
            room_id=str(rid),
            capacity=cap,
            loc_x=loc_x,
            loc_y=loc_y,
            sharing_pattern=pattern,
            sharing_unit=unit,
            initiative=initiative,
            term=term
        ))

# Classes (+ nested)
classes_node = root.find("classes")
if classes_node is not None:
    for c in classes_node.findall("class"):
        cid = c.get("id")
        classes.append(ClassRec(
            class_id=str(cid),
            offering=c.get("offering"),
            config=c.get("config"),
            subpart=c.get("subpart"),
            parent_class_id=c.get("parent"),
            class_limit=attr_int(c, "classLimit"),
            nr_rooms=attr_int(c, "nrRooms"),
            scheduler_dept=c.get("scheduler"),
            committed=parse_bool(c.get("committed")),
            dates_mask=c.get("dates")
        ))
        for t in c.findall("time"):
            cand_times.append(CandidateTime(
                class_id=str(cid),
                days=t.get("days"),
                start=attr_int(t, "start", 0),
                length=attr_int(t, "length", 0),
                pref=attr_float(t, "pref", None)
            ))
        for r in c.findall("room"):
            cand_rooms.append(CandidateRoom(
                class_id=str(cid),
                room_id=str(r.get("id")),
                pref=attr_float(r, "pref", None)
            ))
        for ins in c.findall("instructor"):
            class_instructors.append(ClassInstructor(
                class_id=str(cid),
                instructor_id=str(ins.get("id"))
            ))

print(f"Parsed rooms: {len(rooms)}")
print(f"Parsed classes: {len(classes)}")
print(f"Parsed candidate times: {len(cand_times)}")
print(f"Parsed candidate rooms: {len(cand_rooms)}")
print(f"Parsed class instructors: {len(class_instructors)}")


Parsed rooms: 81
Parsed classes: 521
Parsed candidate times: 1709
Parsed candidate rooms: 609
Parsed class instructors: 33


In [ ]:
# Cell 5 — DataFrames for inspection
# Comment: Convert parsed lists to pandas DataFrames for quick checks.

df_rooms = pd.DataFrame([asdict(r) for r in rooms])
df_classes = pd.DataFrame([asdict(c) for c in classes])
df_cand_times = pd.DataFrame([asdict(ct) for ct in cand_times])
df_cand_rooms = pd.DataFrame([asdict(cr) for cr in cand_rooms])
df_class_instr = pd.DataFrame([asdict(ci) for ci in class_instructors])

df_context = pd.DataFrame([{
    "initiative": initiative,
    "term": term,
    "nr_days": nr_days,
    "slots_per_day": slots_per_day
}])

print(df_context)
print(df_rooms[["room_id","capacity","loc_x","loc_y"]].head())
print(df_classes.head())
print(df_cand_times.head())
print(df_cand_rooms.head())
print(df_class_instr.head())


             initiative     term  nr_days  slots_per_day
0  puWestLafayetteTrdtn  2007Spr        7            288
  room_id  capacity  loc_x  loc_y
0      81        22    450    449
1      82        25    450    449
2      83        24    450    449
3      84        61    450    449
4      85        36    443    458
  class_id offering config subpart parent_class_id  class_limit min_limit  \
0     1244      138    138     766            None         22.0      None   
1     1245      138    138     766            None         22.0      None   
2     1246      138    138     766            None         22.0      None   
3     1247      138    138     766            None         22.0      None   
4     1248      138    138     766            None         22.0      None   

  max_limit  nr_rooms scheduler_dept  committed  \
0      None       0.0              4      False   
1      None       0.0              4      False   
2      None       0.0              4      False   
3      None    

In [ ]:
# Cell 6 — Save normalized tables (CSV)
# Comment: Persist normalized datasets to OUTPUT_DIR for downstream usage.

df_context.to_csv(OUTPUT_DIR / "context.csv", index=False)
df_rooms.to_csv(OUTPUT_DIR / "rooms.csv", index=False)
df_classes.to_csv(OUTPUT_DIR / "classes.csv", index=False)
df_cand_times.to_csv(OUTPUT_DIR / "candidate_times.csv", index=False)
df_cand_rooms.to_csv(OUTPUT_DIR / "candidate_rooms.csv", index=False)
df_class_instr.to_csv(OUTPUT_DIR / "class_instructors.csv", index=False)

print("Saved normalized tables to:", OUTPUT_DIR)


Saved normalized tables to: /mnt/data/unittime_goal1_out


In [ ]:
# Cell 7 — Build Task A (per-class) JSONL
# Comment: One JSON line per class with candidate times/rooms/instructors.

def build_taskA_samples(df_classes, df_cand_times, df_cand_rooms, df_class_instr, nr_days, slots_per_day, df_rooms):
    times_by_class = df_cand_times.groupby("class_id") if not df_cand_times.empty else {}
    rooms_by_class = df_cand_rooms.groupby("class_id") if not df_cand_rooms.empty else {}
    instr_by_class = df_class_instr.groupby("class_id") if not df_class_instr.empty else {}

    samples = []
    for _, row in df_classes.iterrows():
        cid = str(row["class_id"])
        tdf = times_by_class.get_group(cid) if hasattr(times_by_class, "groups") and cid in times_by_class.groups else pd.DataFrame(columns=df_cand_times.columns)
        rdf = rooms_by_class.get_group(cid) if hasattr(rooms_by_class, "groups") and cid in rooms_by_class.groups else pd.DataFrame(columns=df_cand_rooms.columns)
        idf = instr_by_class.get_group(cid) if hasattr(instr_by_class, "groups") and cid in instr_by_class.groups else pd.DataFrame(columns=df_class_instr.columns)

        input_obj = {
            "nr_days": int(nr_days),
            "slots_per_day": int(slots_per_day),
            "class_id": cid,
            "subpart": row.get("subpart"),
            "class_limit": int(row["class_limit"]) if pd.notna(row["class_limit"]) else None,
            "dates_mask": row.get("dates_mask"),
            "instructors": [int(x) if str(x).isdigit() else x for x in idf["instructor_id"].tolist()],
            "candidate_times": tdf[["days","start","length","pref"]].to_dict(orient="records"),
            "candidate_rooms": []
        }

        if len(rdf):
            rdf2 = rdf.merge(df_rooms[["room_id","capacity"]], on="room_id", how="left")
            input_obj["candidate_rooms"] = rdf2[["room_id","capacity","pref"]].to_dict(orient="records")

        samples.append({
            "instruction": "Assign a feasible room and time for the class given the candidates and constraints.",
            "input": input_obj,
            "output": {"assignments": []}  # fill later if gold solutions exist
        })
    return samples

taskA = build_taskA_samples(df_classes, df_cand_times, df_cand_rooms, df_class_instr, nr_days, slots_per_day, df_rooms)
taskA_path = OUTPUT_DIR / "taskA.jsonl"
with open(taskA_path, "w", encoding="utf-8") as f:
    for ex in taskA:
        f.write(json.dumps(ex, ensure_ascii=False) + "\n")
print(f"Wrote Task A: {taskA_path} (count={len(taskA)})")


Wrote Task A: /mnt/data/unittime_goal1_out/taskA.jsonl (count=521)


In [ ]:
# Cell 8 — Build Task B (per-offering) JSONL
# Comment: One JSON line per offering with all its classes & candidates.

def build_taskB_samples(df_classes, df_cand_times, df_cand_rooms, df_class_instr, nr_days, slots_per_day, df_rooms):
    times_by_class = df_cand_times.groupby("class_id") if not df_cand_times.empty else {}
    rooms_by_class = df_cand_rooms.groupby("class_id") if not df_cand_rooms.empty else {}
    instr_by_class = df_class_instr.groupby("class_id") if not df_class_instr.empty else {}

    samples = []
    for offering, grp in df_classes.groupby("offering"):
        classes_payload = []
        for _, row in grp.iterrows():
            cid = str(row["class_id"])
            tdf = times_by_class.get_group(cid) if hasattr(times_by_class, "groups") and cid in times_by_class.groups else pd.DataFrame(columns=df_cand_times.columns)
            rdf = rooms_by_class.get_group(cid) if hasattr(rooms_by_class, "groups") and cid in rooms_by_class.groups else pd.DataFrame(columns=df_cand_rooms.columns)
            idf = instr_by_class.get_group(cid) if hasattr(instr_by_class, "groups") and cid in instr_by_class.groups else pd.DataFrame(columns=df_class_instr.columns)

            cls_obj = {
                "class_id": cid,
                "subpart": row.get("subpart"),
                "class_limit": int(row["class_limit"]) if pd.notna(row["class_limit"]) else None,
                "dates_mask": row.get("dates_mask"),
                "instructors": [int(x) if str(x).isdigit() else x for x in idf["instructor_id"].tolist()],
                "candidate_times": tdf[["days","start","length","pref"]].to_dict(orient="records"),
                "candidate_rooms": []
            }
            if len(rdf):
                rdf2 = rdf.merge(df_rooms[["room_id","capacity"]], on="room_id", how="left")
                cls_obj["candidate_rooms"] = rdf2[["room_id","capacity","pref"]].to_dict(orient="records")

            classes_payload.append(cls_obj)

        samples.append({
            "instruction": "Assign feasible rooms and times for all classes in this offering.",
            "input": {
                "offering_id": str(offering),
                "nr_days": int(nr_days),
                "slots_per_day": int(slots_per_day),
                "classes": classes_payload
            },
            "output": {"assignments": []}
        })
    return samples

taskB = build_taskB_samples(df_classes, df_cand_times, df_cand_rooms, df_class_instr, nr_days, slots_per_day, df_rooms)
taskB_path = OUTPUT_DIR / "taskB.jsonl"
with open(taskB_path, "w", encoding="utf-8") as f:
    for ex in taskB:
        f.write(json.dumps(ex, ensure_ascii=False) + "\n")
print(f"Wrote Task B: {taskB_path} (count={len(taskB)})")


Wrote Task B: /mnt/data/unittime_goal1_out/taskB.jsonl (count=39)


In [ ]:
# Cell 9 — Basic validations
# Comment: Simple sanity checks (e.g., candidate room capacity ≥ class limit).

def hhmm_from_slot(slot: int, slot_len_min=5):
    total_min = slot * slot_len_min
    hh = total_min // 60
    mm = total_min % 60
    return f"{hh:02d}:{mm:02d}"

if not df_cand_rooms.empty and "class_limit" in df_classes.columns:
    class_limits = df_classes.set_index("class_id")["class_limit"].to_dict()
    rooms_cap = df_rooms.set_index("room_id")["capacity"].to_dict()
    feas_rows = []
    for _, r in df_cand_rooms.iterrows():
        cid, rid = r["class_id"], r["room_id"]
        limit = class_limits.get(cid)
        cap = rooms_cap.get(rid)
        feas = None if (limit is None or cap is None) else (cap >= limit)
        feas_rows.append({"class_id": cid, "room_id": rid, "capacity_ok": feas})
    pd.DataFrame(feas_rows).to_csv(OUTPUT_DIR / "room_capacity_flags.csv", index=False)
    print("Saved capacity feasibility flags.")
print("Validation complete.")


Saved capacity feasibility flags.
Validation complete.


In [ ]:
# Cell 10 — Preview JSONL
# Comment: Print a couple of examples from Task A and Task B to verify structure.

from itertools import islice

def peek_jsonl(path, k=2):
    out = []
    with open(path, "r", encoding="utf-8") as f:
        for ln in islice(f, k):
            out.append(json.loads(ln))
    return out

print("Task A preview:")
print(peek_jsonl(taskA_path, 2))

print("\nTask B preview:")
print(peek_jsonl(taskB_path, 2))


Task A preview:
[{'instruction': 'Assign a feasible room and time for the class given the candidates and constraints.', 'input': {'nr_days': 7, 'slots_per_day': 288, 'class_id': '1244', 'subpart': '766', 'class_limit': 22, 'dates_mask': '00000000000000000000000000000000000000111111001111101111110111111011111101111110111111011111101111110000000011111101111110111111011111101111110111111', 'instructors': [], 'candidate_times': [{'days': '0000100', 'start': 90, 'length': 12, 'pref': 0.0}, {'days': '0000100', 'start': 102, 'length': 12, 'pref': 0.0}, {'days': '0000100', 'start': 114, 'length': 12, 'pref': 0.0}, {'days': '0000100', 'start': 126, 'length': 12, 'pref': 0.0}, {'days': '0000100', 'start': 138, 'length': 12, 'pref': 0.0}, {'days': '0000100', 'start': 150, 'length': 12, 'pref': 0.0}, {'days': '0000100', 'start': 162, 'length': 12, 'pref': 0.0}, {'days': '0000100', 'start': 174, 'length': 12, 'pref': 0.0}, {'days': '0000100', 'start': 186, 'length': 12, 'pref': 0.0}, {'days': '0000

In [ ]:
###goal 2

In [ ]:
# Cell 1 — Install dependencies
# Comment: Transformers + TRL + PEFT + 4-bit quantization stack.

!pip -q install --upgrade transformers accelerate datasets peft bitsandbytes trl evaluate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 86.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.9/504.9 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.6 MB/s eta 0:00:00


In [ ]:
# Cell 2 — Imports & configuration
# Comment: Paths, base model, basic run settings.

from pathlib import Path
import json, random, os, torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig
from trl import SFTTrainer, SFTConfig

# Paths
DATA_DIR = Path("/mnt/data/unittime_goal1_out")
TASKA = DATA_DIR / "taskA.jsonl"
TASKB = DATA_DIR / "taskB.jsonl"
MODEL_DIR = Path("/mnt/data/unittime_goal2_models")
MODEL_DIR.mkdir(parents=True, exist_ok=True)

assert TASKA.exists(), f"Missing {TASKA}; run Goal 1 first."
assert TASKB.exists(), f"Missing {TASKB}; run Goal 1 first."

# Base model (small, chat-tuned)
BASE_MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# Seed
SEED = 42
random.seed(SEED); torch.manual_seed(SEED)

print(f"Base model: {BASE_MODEL}")
print(f"Data: {TASKA.name}, {TASKB.name}")
print(f"Output dir: {MODEL_DIR}")


Base model: TinyLlama/TinyLlama-1.1B-Chat-v1.0
Data: taskA.jsonl, taskB.jsonl
Output dir: /mnt/data/unittime_goal2_models


In [ ]:
# Cell 3 — Load datasets & split
# Comment: Read JSONL into HF Datasets and make a 90/10 split.

def load_jsonl(path: Path):
    rows = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            rows.append(json.loads(line))
    return Dataset.from_list(rows)

dsA = load_jsonl(TASKA)
dsB = load_jsonl(TASKB)

def train_val_split(ds, val_ratio=0.1, seed=SEED):
    n = len(ds)
    val_size = max(1, int(n * val_ratio))
    ds = ds.shuffle(seed=seed)
    return ds.select(range(n - val_size)), ds.select(range(n - val_size, n))

trainA, valA = train_val_split(dsA, 0.1)
trainB, valB = train_val_split(dsB, 0.1)

print("Stage A:", len(trainA), "train,", len(valA), "val")
print("Stage B:", len(trainB), "train,", len(valB), "val")


Stage A: 469 train, 52 val
Stage B: 36 train, 3 val


In [ ]:
# Cell 4 — Prompt formatting
# Comment: Turn each JSON example into a single training "text" for causal LM.

SYSTEM_PROMPT = "You are a helpful timetable assistant. Always return strict JSON."

def format_example(ex):
    instruction = ex.get("instruction", "").strip()
    input_obj = ex.get("input", {})
    output_obj = ex.get("output", {})
    input_json = json.dumps(input_obj, ensure_ascii=False, sort_keys=True)
    output_json = json.dumps(output_obj, ensure_ascii=False, sort_keys=True)
    text = (
        f"<|system|>\n{SYSTEM_PROMPT}\n"
        f"<|user|>\n{instruction}\nINPUT:\n{input_json}\n"
        f"<|assistant|>\n{output_json}"
    )
    return {"text": text}

trainA_f = trainA.map(format_example)
valA_f   = valA.map(format_example)
trainB_f = trainB.map(format_example)
valB_f   = valB.map(format_example)

print(trainA_f[0]["text"][:500])


Map:   0%|          | 0/469 [00:00<?, ? examples/s]

Map:   0%|          | 0/52 [00:00<?, ? examples/s]

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

<|system|>
You are a helpful timetable assistant. Always return strict JSON.
<|user|>
Assign a feasible room and time for the class given the candidates and constraints.
INPUT:
{"candidate_rooms": [{"capacity": 136, "pref": 0.0, "room_id": "31"}], "candidate_times": [{"days": "1010000", "length": 18, "pref": 0.0, "start": 162}], "class_id": "626", "class_limit": null, "dates_mask": "000000000000000000000000000000000000001111110011111011111101111110111111011111101111110111111011111100000000111111


In [ ]:
# Cell 5 — Load model in 4-bit & setup LoRA
# Comment: QLoRA config + tokenizer + base model.

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
)

peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# Cell 6 — Stage A training (per-class)  ✅ VERSION-SAFE

output_dir_A = (MODEL_DIR / "tinyllama_qlora_taskA").as_posix()

# Keep SFTConfig minimal & compatible across TRL versions
train_cfg_A = SFTConfig(
    output_dir=output_dir_A,
    num_train_epochs=1,                   # increase after a smoke test
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    logging_steps=20,
    bf16=torch.cuda.is_available(),
    fp16=not torch.cuda.is_available(),
    optim="paged_adamw_32bit",
    eval_strategy="steps",
    eval_steps=100,
    save_steps=200,
    save_total_limit=2,
    packing=False,                       # leave seq len handling to trainer/defaults
    dataset_text_field="text",
)

# Build the trainer with graceful fallbacks for different TRL signatures
trainerA = None
last_err = None
for kwargs in [
    # Old TRL: accepts 'tokenizer'
    dict(tokenizer=tokenizer),
    # Newer TRL: no 'tokenizer', accepts max_seq_length on trainer
    dict(max_seq_length=2048),
    # Fallback: minimal
    dict(),
]:
    try:
        trainerA = SFTTrainer(
            model=model,
            train_dataset=trainA_f,
            eval_dataset=valA_f,
            peft_config=peft_config,
            args=train_cfg_A,
            **kwargs
        )
        break
    except TypeError as e:
        last_err = e

if trainerA is None:
    raise last_err

trainerA.train()
trainerA.save_model(output_dir_A)     # saves LoRA adapter
tokenizer.save_pretrained(output_dir_A)
print(f"Saved Stage A adapter → {output_dir_A}")


Adding EOS to train dataset:   0%|          | 0/469 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/469 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/469 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/52 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/52 [00:00<?, ? examples/s]

...

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


CommError: Error uploading run: returned error 401: {"data":{"upsertBucket":null},"errors":[{"message":"user is not logged in","path":["upsertBucket"],"extensions":{"code":"PERMISSION_ERROR"}}]}